In [128]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb, perm
from collections import defaultdict
from dataclasses import dataclass
from bisect import insort
import string

files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [134]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))
lines = sum([re.split(':|\.', line) for line in lines], [])

p_ore = re.compile('(\d+) ore')
p_clay = re.compile('(\d+) clay')
p_obsidian = re.compile('(\d+) obsidian')

B = []
M = []
i = 1
while i < len(lines):
    b = {}
    m = (0,0,0)
    for j, x in enumerate(['ore', 'clay', 'obsidian', 'geode']):
        ore, clay, obsidian = (0, 0, 0)
        pat = p_ore.search(lines[i+j])
        if pat:
            ore = int(pat.groups()[0])
        pat = p_clay.search(lines[i+j])
        if pat:
            clay = int(pat.groups()[0])
        pat = p_obsidian.search(lines[i+j])
        if pat:
            obsidian = int(pat.groups()[0])
        b[x] = (ore, clay, obsidian)
        m = (max(m[0], ore), max(m[1], clay), max(m[2], obsidian))
    B.append(b)
    M.append(m)
    i += 6

print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [135]:
@dataclass
class Resources:
    ore: int
    clay: int
    obsidian: int
    geode: int

    def __hash__(self):
        return hash((self.ore, self.clay, self.obsidian, self.geode))


@dataclass
class Robots:
    ore: int
    clay: int
    obsidian: int
    geode: int

    def __hash__(self):
        return hash((self.ore, self.clay, self.obsidian, self.geode))


class State:
    def __init__(self, t: int, robots: Robots, resources: Resources):
        self.t = t
        self.robots = robots
        self.resources = resources

    def produce(self):
        self.resources.ore += self.robots.ore
        self.resources.clay += self.robots.clay
        self.resources.obsidian += self.robots.obsidian
        self.resources.geode += self.robots.geode

    def branch(self):
        branches = []
        blacklist = []
        for name in self.possible_builds():
            b = deepcopy(self)
            b.build(name)
            b.wait([])
            setattr(b.resources, name, getattr(b.resources, name) - 1)
            branches.append(b)
            blacklist.append(name)
        wait = deepcopy(self)
        wait.wait(blacklist)
        branches.append(wait)
        return branches

    def wait(self, blacklist):
        while True:
            self.t -= 1
            self.produce()
            if self.t == 0:
                break
            if len(set(self.possible_builds()) - set(blacklist)) > 0:
                break

    def possible_builds(self):
        builds = []
        for name, (ore, clay, obsidian) in list(State.blueprint.items())[::-1]:
            if ore <= self.resources.ore \
                    and clay <= self.resources.clay\
                    and obsidian <= self.resources.obsidian\
                    and self.robots.ore <= State.maximum[0]\
                    and self.robots.clay <= State.maximum[1]\
                    and self.robots.obsidian <= State.maximum[2]:
                builds.append(name)
                if name == 'geode':
                    break
                if name == 'obsidian' and self.t < 15:
                    break

        return builds

    def build(self, name):
        ore, clay, obsidian = State.blueprint[name]
        self.resources.ore -= ore
        self.resources.clay -= clay
        self.resources.obsidian -= obsidian
        setattr(self.robots, name, getattr(self.robots, name) + 1)

    def __lt__(self, other):
        if self.t == other.t:
            return self.robots.obsidian < other.robots.obsidian
        return self.t < other.t

    def __hash__(self):
        return hash((self.t, self.robots, self.resources))

    def __repr__(self):
        return f"t={self.t}\nrobots={self.robots}\nresources={self.resources}\n"


In [136]:
def quality(i, time, blueprint, maximum):
    State.blueprint = blueprint
    State.maximum = maximum
    robots = Robots(1, 0, 0, 0)
    resources = Resources(0, 0, 0, 0)
    START = State(time, robots, resources)
    Q = [START]
    V = set()
    maxg = 0

    while len(Q) > 0:
        state = Q.pop()
        if state.resources.geode < maxg:
            continue
        maxg = max(maxg, state.resources.geode)

        if state in V:
            continue
        else:
            V.add(state)

        if state.t == 0:
            continue

        for b in state.branch():
            insort(Q, b)

    return maxg * i


In [137]:
import multiprocess as mp

with mp.Pool(5) as pool:
    results = pool.starmap(
        quality, [(i+1, 24, B[i], M[i]) for i in range(len(B))])

print(sum(results))


2193


### Part 2

In [138]:
import multiprocess as mp

N = 2 if box.value == "ex1.txt" else 3
with mp.Pool(5) as pool:
    results = pool.starmap(quality, [(1, 32, B[i], M[i]) for i in range(N)])

print(prod(results))


7200
